In [1]:
import pandas as pd
from pathlib import Path
import re
import pandas as pd
import tqdm

PATH_TO_FILES=Path('C:\Datalab\sida_projekt\sida_projekt\Riksdagsdata\Propositioner')
PATH_TO_OUTDATA=Path('data')

# Skippa dokument med titlar som innehåller:
FILTER_TITLE='vårproposition|budgetproposition|ändringsbudget' # Lämna tom sträng om du inte vill filtrera


In [8]:
def read_to_df(PATH_TO_FILES=PATH_TO_FILES,FILTER_TITLE=''):
    
    # Read metadata
    df1=pd.read_csv(PATH_TO_FILES / 'prop-2018-2021.csv',header=None)
    df2=pd.read_csv(PATH_TO_FILES /'prop-2014-2017.csv',header=None)
    df=pd.concat([df1,df2])
    print('Raw input shape', df.shape)

    #Pre-processing
    df.loc[:,1]=df.loc[:,1].str.lower()
    df.set_index(1, inplace=True)
    df.index.rename('filename', inplace=True)
    df=df.rename(columns={2:'rm',8:'Departement',9:'Utskott',13:'Titel'})

    # Filter out
    df=df.loc[~df['Titel'].str.contains(FILTER_TITLE,flags=re.I),['Titel','rm','Departement','Utskott']]
    print('Output shape:',df.shape)
    return df

def add_space(text):
    """ Adds space before (suspected) chapter"""
    return re.sub('\.\s*(\n[A-Ö\d][\.]?\w*:?( [\w\-–:]*){0,7}\s*?\n+)[A-Ö]',r'\n\1',text)

def remove_chapnumber(text):
    """Remove chapter number
    Returns cleaned text"""
    return re.sub('\n\d+\.\d+\n|\n\d+\n','',text)

def avstava(text):
    
    # Ta bort sidnummer om det kommer in i en avstavning
    text=re.sub('([a-ö]{2,}-\s*)(\d+)',r'\1',text)

    streck=re.compile(r"""  (?!it)          # Matcha inte på it
                            (               # Grupp 1 innehåller grupp 2 till 4:
                                ([a-ö]{2,})     # Grupp 2 första delen av det eventuellt avstavade ordet (kräver minst 2 bokstäver)
                                (-\s*\d*\s*)    # Grupp 3 vill vi ta bort från hela grupp 0 (fångar även insprängda sidnummer)
                                ([a-ö]*)        # Grupp 4 Sista delen i det eventuellt avstavade ordet
                                )"""            # Notera att vi, i grupp 4, endast matchar på små bokstäver från a till ö
        ,                                       
        re.X + re.U
    )       
        
    def dashrepl(matchobj):
        """ Fixar avstavningar """
        if matchobj.group(4) is None:
            return matchobj.group(0)
        elif matchobj.group(2).islower():
            if (matchobj.group(2) in ['bnp','tfp']) or (matchobj.group(4) in ['och','eller']): # Manuella tillägg av godkända prefix och ord efter bindesstrecket som är tillåtna.
                return matchobj.group(0)
            else:
                return matchobj.group(2) + matchobj.group(4)
        else:
            return matchobj.group(0) 

    return re.sub(streck,dashrepl,text)

def cleaner(text):
    """Chained cleaner functions"""
    return add_space(remove_chapnumber(avstava(text)))

def import_and_clean_txt(df, PATH_TO_FILES=PATH_TO_FILES,clean=True):
    for ids, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:    
            with open(PATH_TO_FILES / (ids + '.txt'), encoding='utf8') as file:
                if clean:
                    df.at[ids,'text']=cleaner(file.read())
                else:
                    df.at[ids,'text']=file.read()
        except FileNotFoundError as ex:
            print(ex)
            print(ids, row['Titel'])
            df.at[ids,'text']='' # File not found
    print('Tomma dokument')
    print(df.loc[df.text==''])
    print('Tar bort tomma dokument...')

    return df.loc[~(df.text=='')]

In [ ]:
df=read_to_df(PATH_TO_FILES=PATH_TO_FILES,FILTER_TITLE=FILTER_TITLE)
df=import_and_clean_txt(df)
print('df.shape:',df.shape)

In [35]:
# Pickle vs. parquet save: 2.5s vs. 4.8s
df.to_pickle(PATH_TO_OUTDATA / 'data.pkl')

# Pickle vs. parquet load: 1.8 vs. 3.6
# df_pqt=pd.read_pickle(PATH_TO_OUTDATA / 'data.pkl')